# DIY Covid-19 Dashboard

I began building my COVID-19 dashboard by importing the necessary libraries for my dashboard to work and display interactive features. Here the ipywidgets library is used to create the interactive interface, pandas is used for analysng the data imported from PHE, matplotlib is used to plot this data on a graph, json is used to store initial data from PHE, while datetime and timedelta are used to provide a timeframe selection tool for the dashboard.

In [ ]:
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
from datetime import datetime, timedelta

In [ ]:
%matplotlib inline
# making figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

The followig function loads the json file, containing the initial data. This data is used to display default values, when the dashboard is loaded.

In [ ]:
# Loading JSON files and storing the raw data in a variable
def load_initial_data_file():
    initialdata = "initialdata.json"
    with open(initialdata, "rt") as INFILE:
        return json.load(INFILE)["data"]

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [ ]:
def wrangle_data(rawdata):
    df = pd.DataFrame(rawdata)
    # Converting the date column to datetime objects, useful for sorting dates and manipulating the dataseries. This is needed as the initial data is strings.
    df["date"] = pd.to_datetime(df["date"])
    # Sorting the values in the dataframe by date
    # inplace=True, replaces the old dataframe with the updated one
    df.sort_values(by="date", inplace=True)
    return df

initial_data = load_initial_data_file()
# Wrangling the new data - sorting the data by date and converting it to a datetime object
df = wrangle_data(initial_data)

## Downloading current data and creating a 'Refresh' button

The following code downloads the custom data from the PHE API and creates a button, which forces ipywidgets to redraw the graph, when the user presses the 'Refresh' button, or when the user changes the data type he/she wants to analyse. 

In [ ]:
def access_api():
    filters = [
        "areaType=nation",
        "areaName=england"
    ]
    # Defining the required data types to be fetched from the PHE database
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospitalcases": "hospitalCases",
        "deaths": "cumDeaths28DaysByPublishDate",
        "newtests": "newTestsByPublishDate",
        "newvaccinations": "newVaccinesGivenByPublishDate",
        "newreinfections": "newReinfectionsBySpecimenDate"
    }
    # Creating an API object 
    api = Cov19API(filters=filters, structure=structure)
    data = api.get_json()
    return data

In [ ]:
# Creating the button, naming it and adjustiing the layout
def api_button_callback(button):
    global df
    apidata = access_api()
    df = wrangle_data(apidata["data"])
    refresh_graph()
    apibutton.icon = "check"

apibutton = wdg.Button(
    description="Refresh", 
    disabled=False,
    button_style="success", 
    tooltip="Click to refresh the data",
    icon="refresh",
    layout=wdg.Layout(width='auto')
)
apibutton.on_click(api_button_callback)

# Mapping metric names to user friendly names for display and creating dropdown menus

In the current setup, the API-fetched metric names are displayed in their original format (in lowercase letters, with no spacing). In order to make the dashboard more professional looking, I will implement a metric_mapping function to change the names displayed.

Further, dropdown menus are added to enable the user to select the data types which he/she would like to analyse, as well as a timeframe selector to enable the user to filter the data for 1 day, 1 week, 1 month and 1 year.

In [ ]:
# Mapping API metrics to user-readable cvalues
metric_mapping = {
    "date": "Date",
    "cases": "New Cases",
    "hospitalcases": "Hospital Admissions",
    "deaths": "Deaths",
    "newtests": "New Tests",
    "newvaccinations": "New Vaccinations",
    "newreinfections": "New Reinfections"
}

# Timeframe selector
timeframe_options = {
    "1 day": pd.Timedelta(days=1),
    "1 week": pd.Timedelta(weeks=1),
    "1 month": pd.Timedelta(days=30),
    "1 year": pd.Timedelta(days=365),
    "all time": pd.Timedelta(days=10000)
}
parameter1_dropdown = wdg.Dropdown(
    options=list(metric_mapping.values()), 
    description="Select Data Type 1:", 
    value="New Cases"
)
parameter2_dropdown = wdg.Dropdown(
    options=list(metric_mapping.values()), 
    description="Select Data Type 2:", 
    value="Deaths"
)
timeframe_dropdown = wdg.Dropdown(
    options=list(timeframe_options.keys()), 
    description="Select Timeframe:", 
    value="all time"
)


## Visualising the data

The following is implemeented to visualise the data on graph, with two y-axis scales, used for comparing data which has radically different value ranges.


In [ ]:
def plot_data(parameter1, parameter2, timeframe):
    if not df.empty:
        end_date = df["date"].max()
        start_date = end_date - timeframe_options[timeframe]
        filtered_df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

        param1_col = next(key for key, value in metric_mapping.items() if value == parameter1)
        param2_col = next(key for key, value in metric_mapping.items() if value == parameter2)

        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting the first data type and assinging a color to differentiate the different sides for easier readability
        color1 = "tab:blue"
        ax1.set_xlabel("Date")
        ax1.set_ylabel(metric_mapping[param1_col], color=color1)
        ax1.plot(filtered_df["date"], filtered_df[param1_col], label=parameter1, color=color1)
        ax1.tick_params(axis="y", labelcolor=color1)

        # Creating a second y-axis the second data type
        # This is used for graphs, where the values of one of the selected data types are significantly different from the other
        # For example, if vaccination numbers range, on average from 0 to 100, while the cases count is in the thousands, a separate axis scale is required
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color2 = "tab:red"
        ax2.set_ylabel(metric_mapping[param2_col], color=color2) 
        ax2.plot(filtered_df["date"], filtered_df[param2_col], label=parameter2, color=color2)
        ax2.tick_params(axis="y", labelcolor=color2)

        # Adjusting the layout to avoid clipping of the right axis
        fig.tight_layout()  
        plt.title("COVID-19 Data Comparison")
        plt.show()
    else:
        print("Data not available or failed to load.")


    
# Refresh graph function which forces ipywidgets to update the graph, when the button is pressed.
def refresh_graph():
    plot_data(parameter1_dropdown.value, parameter2_dropdown.value)

# Creating an interactive output for the graph
graph = wdg.interactive_output(plot_data, {
    "parameter1": parameter1_dropdown, 
    "parameter2": parameter2_dropdown,
    "timeframe": timeframe_dropdown
})

## Dashboard with refresh button


In [ ]:
def on_param_change(change):
    plot_data(parameter1_dropdown.value, parameter2_dropdown.value, timeframe_dropdown.value)

def on_param_change(change):
    refresh_graph()
        
parameter2_dropdown.observe(on_param_change, names="value")
timeframe_dropdown.observe(on_param_change, names="value")

In [ ]:
display(apibutton)
display(wdg.HBox([parameter1_dropdown, parameter2_dropdown, timeframe_dropdown]))
display(graph)

**Author and Copyright Notice** Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Ivan Elagin, 2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).